<img style="float: center;" src="https://raw.githubusercontent.com/actinia-org/actinia-core/main/docs/docs/actinia_logo.svg" width="25%">

## Einführung

Die cloudbasierte Geoprocessing-Plattform [actinia](https://actinia.mundialis.de/)
ist in der Lage, große Mengen von Geodaten in der Cloud zu verarbeiten und zu
analysieren. Die Software vereinigt viele [GRASS GIS](https://grass.osgeo.org/)
Verarbeitungswerkzeuge und Datenbanken als
[REST-Dienst](https://en.wikipedia.org/wiki/Representational_State_Transfer).
Daher ist der Zugriff auf GRASS-Ressourcen wie Rasterkarten, Raum-Zeit-Rasterdatensätze,
Verarbeitungs- und Analysemodule über URLs möglich. Darüber hinaus ermöglicht
actinia die cloudbasierte Verarbeitung von Daten, zum Beispiel alle Landsat 4-9 Szenen
sowie alle Sentinel-2-Szenen in einer ephemeren Datenbank. Die Berechnungsergebnisse
stehen in der ephemeren Verarbeitung über Objektspeicher als GeoTIFF/COG Rasterdateien oder
GeoPackage-Vektordateien zur Verfügung.

Der actinia-Dienst besteht aus dem *[actinia core](https://github.com/actinia-org/actinia-core)*,
der den grundlegenden, aber anspruchsvollen Verarbeitungsdienst bereitstellt, und
*[actinia plugins](https://github.com/orgs/actinia-org/repositories?q=actinia+plugins&type=all&language=&sort=)*,
die themenpezifische Dienste wie NDVI-Berechnungen aus Sentinel-2 oder Landsat-Daten,
räumlich-zeitliche statistische Analysen und vieles mehr anbieten.

Das folgende Beispiel ist eine überarbeitete Jupyter Notebook Version des Online
[actinia Tutorials](https://actinia-dev.mundialis.de/tutorial/introduction.html).

### Was ist REST?

Der Representational State Transfer ([REST](https://de.wikipedia.org/wiki/Representational_State_Transfer))
ist eine Abstraktion der Struktur und des Verhaltens des World Wide Web
([HTTP](https://de.wikipedia.org/wiki/Hypertext_Transfer_Protocol)). Ziel von REST
ist es, einen Architekturstil zu schaffen, der den Anforderungen des modernen Web
besser gerecht wird. Dabei unterscheidet sich REST von anderen Architekturstilen
vor allem durch die Forderung nach einer einheitlichen Schnittstelle. Der Fokus von
REST liegt auf der Maschine-zu-Maschine-Kommunikation. REST benutzt die
"[request methods](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Request_methods)"
GET, DELETE, POST und PUT, um Ressourcen mit zustandslosen Operationen zu manipulieren
und zu empfangen.

Während GET-Anfragen einfach von einem Browser gesendet werden können, sind POST-,
PUT- oder DELETE-Anfragen nicht möglich. Um das volle Potenzial von actinia zu nutzen,
benötigen Sie einen HTTP-Client, der alle HTTP-Kommunikationsmethoden beherrscht.
Für diesen Zweck werden wir hier dieses Jupyter Notebook verwenden.

### GRASS GIS Data Management

GRASS GIS, was intern neben GDAL und anderen Komponenten von actinia verwendet wird, hat die in der Abbildung dargestellte Datenbankstruktur.

![grass_format](images/grass_format.png)

**Funktionalität über GRASS GIS hinaus**

Actinia ist nicht nur eine REST-Schnittstelle zu GRASS GIS, sondern bietet die Möglichkeit, seine Funktionalität mit anderer Software (ESA SNAP, GDAL, ...) zu erweitern. Um andere als GRASS GIS Software zu integrieren, muss ein Wrapper-Skript geschrieben werden (Stil: als GRASS GIS Addon Python-Skript), das dann die entsprechenden Funktionsaufrufe der zu integrierenden Software enthält. Der Aufruf von Shell-Befehlen in einer actinia-Prozesskette ist ebenfalls möglich, aber aufgrund von Sicherheitsrisiken eingeschränkt.


### Persistent vs. User Database

Unter **Persistentem Speicher** versteht man einen Datenspeicher, der die Daten auch bei einem Stromausfall und ohne geplante Löschzeiten aufbewahrt. Im Geo/EO-Kontext wird der persistente Speicher z. B. für die Bereitstellung von Basiskartographie verwendet, d. h. für Höhenmodelle, Straßennetze, Gebäudegrundrisse usw.

Der **ephemere Speicher** wird für auf Anfrage berechnete Ergebnisse verwendet, einschließlich benutzergenerierter Daten und temporärer Daten, die in Verarbeitungsketten anfallen. In einem ephemeren Speicher werden die Daten nur für einen begrenzten Zeitraum aufbewahrt (z. B. in Actinia standardmäßig für 24 Stunden).

Im Kontext des Cloud Computing sind diese Unterschiede relevant, da bei der Speicherung von Daten Kosten anfallen.

Dementsprechend bietet actinia zwei Betriebsmodi an: persistente und ephemere (User) Verarbeitung. Insbesondere wird der **actinia server** typischerweise auf einem Server mit Zugriff auf eine persistente GRASS GIS Datenbank (PDB) und optional auf eine oder mehrere GRASS GIS Nutzerdatenbanken (UDB) eingesetzt.

---

### actinia API documentation

Die Schnittstellen Dokumentation findet sich hier:

* [actinia "stable" API v3 docs](https://redocly.github.io/redoc/?url=https://actinia.mundialis.de/api/v3/swagger.json)
* [actinia "development" API v3 docs](https://redocly.github.io/redoc/?url=https://actinia-dev.mundialis.de/api/v3/swagger.json)

---

### Anforderungen

#### Software & Module

Dieses Tutorial setzt voraus, dass Sie mit der Programmiersprache [Python](https://python.org) vertraut sind. Die Kenntnis grundlegender REST-API-Konzepte und deren Verwendung wird ebenfalls vorausgesetzt.

Die in diesem Tutorium verwendeten Python-Module sind:
* [requests](http://docs.python-requests.org/)
* [json](https://docs.python.org/3/library/json.html)
* [leafmap](https://leafmap.org/)

#### Actinia API Benutzer und Passwort

Für diese Demo werden die Anmeldedaten für die Authentifizierung benötigt, die unten in **Vorbereitung** als Variable festgelegt sind. Eine andere Actinia-Instanz kann andere Anmeldedaten erfordern.

### Hilfsmodule und Funktionen

<span style="color:red">**TODO is this needed???**</span> --> MN: please no

Bevor wir mit dem actinia-Server über Python interagieren, werden wir die erforderlichen Pakete importieren und eine Hilfsfunktion einrichten, um formatiertes JSON mit json zu drucken.

***Hinweis:*** Möglicherweise müssen Sie zwei hilfreiche Browser-Plugins namens **RESTman** und **JSON Formatter** installieren, die JSON formatieren und leichter lesbar machen:

* [RESTman-Erweiterung](https://chrome.google.com/webstore/detail/restman/ihgpcfpkpmdcghlnaofdmjkoemnlijdi)
* [JSON Formatter](https://chrome.google.com/webstore/detail/json-formatter/bcjindcccaagfpapjjmafapmmgkkhgoa)

## Vorbereitung

Um diese Sitzung im Jupyter Notebook zu initialisieren, laden wir einige Python Bibliotheken.

In [1]:
# Zuerst werden die erforderlichen Python Bibliotheken importiert.

from pprint import pprint
import sys
import json
import shutil
import time

import requests
from requests.auth import HTTPBasicAuth


Um die Kommunikation mit dem actinia Server zu vereinfachen, speichern wir die Anmeldedaten und die URL des REST-Servers in Variablen:

In [5]:
# variables to set the actinia host, version, and user

actinia_baseurl = "https://actinia.mundialis.de"
actinia_version = "v3"
actinia_url = actinia_baseurl + "/api/" + actinia_version
actinia_auth = HTTPBasicAuth('fossgis2023', 'ieh0ahweefavicieca6g')

In [6]:
# helper function to print formatted JSON using the json module

def print_as_json(data):
    print(json.dumps(data, indent=2))

# helper function to verify a request
def verify_request(request, success_code=200):
    if request.status_code != success_code:
        print("ERROR: actinia processing failed with status code %d!" % request.status_code)
        print("See errors below:")
        print_as_json(request.json())
        request_url = request.json()["urls"]["status"]
        requests.delete(url=request_url, auth=actinia_auth)
        raise Exception("The resource <%s> has been terminated." % request_url)

## Beispiele

* Datenverwaltung
* NDVI-Berechnung mit Landsat-8 und Sentinel-2

### Datenverwaltung

Auflistung aller GRASS locations, die über den Endpunkt `/locations` in der persistenten Datenbank von actinia verfügbar sind:

In [7]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Available locations:")

# print formatted JSON
print_as_json(jsonResponse)

actinia GET request:
https://actinia.mundialis.de/api/v3/locations
---
Available locations:
{
  "locations": [
    "latlong_wgs84",
    "ECAD",
    "nc_spm_08",
    "fossgis2023_epsg25832_utm32N"
  ],
  "status": "success"
}


#### Mapsets in Locations auflisten

Auflisten aller **Mapsets** innerhalb der Location `fossgis2023_epsg25832_utm32N` über den Endpunkt `/locations/<location_name>/mapsets`:

In [8]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/fossgis2023_epsg25832_utm32N/mapsets"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Mapsets in fossgis2023_epsg25832_utm32N location:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

actinia GET request:
https://actinia.mundialis.de/api/v3/locations/fossgis2023_epsg25832_utm32N/mapsets
---
Mapsets in fossgis2023_epsg25832_utm32N location:
[
  "PERMANENT"
]


#### Inhalt einer Mapset auflisten

Auflisten aller **Rasterkarten** in der Location `fossgis2023_epsg25832_utm32N` und Mapset `PERMANENT` über den Endpunkt `/locations/<location_name>/mapsets/<mapset>/raster_layers`:

In [9]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/fossgis2023_epsg25832_utm32N/mapsets/PERMANENT/raster_layers"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Raster layers in mapset PERMANENT of location fossgis2023_epsg25832_utm32N:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

actinia GET request:
https://actinia.mundialis.de/api/v3/locations/fossgis2023_epsg25832_utm32N/mapsets/PERMANENT/raster_layers
---
Raster layers in mapset PERMANENT of location fossgis2023_epsg25832_utm32N:
[
  "S2_NDVI_average",
  "S2_NDVI_maximum",
  "S2_NDVI_stddev",
  "S2_NDWI_average",
  "S2_NDWI_maximum",
  "S2_NDWI_stddev",
  "S2_brightness_average",
  "S2_brightness_maximum",
  "S2_brightness_stddev",
  "dop_blue",
  "dop_green",
  "dop_nir",
  "dop_red",
  "ndom",
  "training_set"
]


Abfrage der **Rasterinformationen** von der Karte `nDOM` über den Endpunkt `/locations/<location_name>/mapsets/<mapset>/raster_layers/<raster>`:

In [10]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/fossgis2023_epsg25832_utm32N/mapsets/PERMANENT/raster_layers/ndom"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Raster info of ndom in mapset PERMANENT of location fossgis2023_epsg25832_utm32N:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

actinia GET request:
https://actinia.mundialis.de/api/v3/locations/fossgis2023_epsg25832_utm32N/mapsets/PERMANENT/raster_layers/ndom
---
Raster info of ndom in mapset PERMANENT of location fossgis2023_epsg25832_utm32N:
{
  "cells": "3577600",
  "cols": "2080",
  "comments": "\"r.in.gdal -a -k -r input=\"/mnt/data/geodata/versiegelungsdaten/ndom_\\bonn/ndom50_tiff_kacheln/ndom_bonn.vrt\" output=\"ndom\" memory=5000 of\\fset=0 num_digits=0\"",
  "creator": "\"mundialis\"",
  "database": "/actinia_core/workspace/temp_db/gisdbase_ca3f20b6b76a45c3ae9ca7de512c742f",
  "datatype": "FCELL",
  "date": "\"Tue Feb 21 09:42:05 2023\"",
  "description": "\"generated by r.in.gdal\"",
  "east": "369350",
  "ewres": "0.5",
  "location": "fossgis2023_epsg25832_utm32N",
  "map": "ndom",
  "mapset": "PERMANENT",
  "maptype": "raster",
  "max": "39.74",
  "min": "-3.310001",
  "ncats": "0",
  "north": "5621100",
  "nsres": "0.5",
  "rows": "1720",
  "semantic_label": "\"none\"",
  "source1": "\"\"",
  "so

**Rendere eine Rasterkarte**: Rendern der Karte `ndom` über den Endpunkt `/locations/<location_name>/mapsets/<mapset>/raster_layers/<raster>/render`:

In [11]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/fossgis2023_epsg25832_utm32N/mapsets/PERMANENT/raster_layers/ndom/render"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth, stream=True)

# # check if anything went wrong
# verify_request(request, 200)

with open('img.png', 'wb') as out_file:
    shutil.copyfileobj(request.raw, out_file)

# # get a json-encoded content of the response
# jsonResponse = request.json()

# print("Raster info of ndom in mapset PERMANENT of location fossgis2023_epsg25832_utm32N:")

# # print formatted JSON
# print_as_json(jsonResponse["process_results"])


# import shutil

# import requests

# url = 'http://example.com/img.png'
# response = requests.get(url, stream=True)
# with open('img.png', 'wb') as out_file:
#     shutil.copyfileobj(response.raw, out_file)

actinia GET request:
https://actinia.mundialis.de/api/v3/locations/fossgis2023_epsg25832_utm32N/mapsets/PERMANENT/raster_layers/ndom/render
---


Die `ndom` Ergebniskarte wird folgendermaßen aussehen:

![requested_ndom_png](images/ndom.png)

Auflistung aller **Vektorkarten** in der Location `fossgis2023_epsg25832_utm32N` und Mapset `PERMANENT` über Endpunkt `/locations/<location_name>/mapsets/<mapset>/vector_layers`:

In [12]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/fossgis2023_epsg25832_utm32N/mapsets/PERMANENT/vector_layers"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Vector layers in mapset PERMANENT of location fossgis2023_epsg25832_utm32N:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

actinia GET request:
https://actinia.mundialis.de/api/v3/locations/fossgis2023_epsg25832_utm32N/mapsets/PERMANENT/vector_layers
---
Vector layers in mapset PERMANENT of location fossgis2023_epsg25832_utm32N:
[
  "alkis_nutzung",
  "buildings",
  "roads",
  "training_set"
]


---

### Benutzerdefinierte Verarbeitung

Actinia nutzt den **Prozesskettenansatz**, um für Import, Verarbeitung und Export von Geodaten mit dem actinia GRASS GIS Verarbeitungssystem zu kommunizieren.

#### Was ist eine Prozesskette?

Eine Prozesskette ist eine Liste von GRASS GIS Modulen, die nacheinander in der Reihenfolge der Liste ausgeführt werden. GRASS GIS Module werden als Prozessdefinitionen spezifiziert, die den Namen des Befehls, die Ein- und Ausgaben, einschließlich Import- und Exportdefinitionen sowie die Modulflags enthalten.

Die Prozesskette muss in JSON formuliert werden.

#### Ephemere vs. persistente Verarbeitung

Die Verarbeitung erfolgt immer in einer temporären ephemeren Datenbank, mit der die Daten aus der persistenten und der Benutzerdatenbank verknüpft werden.

Der Prozess kann dann **ephemeral** oder **persistent** durchgeführt werden.

Bei der **persistenten** Verarbeitung kann die ephemere Datenbank in die persistente Benutzerdatenbank verschoben werden, so dass die Berechnungsergebnisse in weiteren Verarbeitungsschritten verwendet oder mit Hilfe der actinia REST-Aufrufe visualisiert werden können.

Bei der **ephemeren** Verarbeitung wird die ephemere Datenbank nach der Berechnung entfernt.

Allerdings können alle Raster- und Vektordaten, die während der Verarbeitung erzeugt wurden, mit GDAL/OGR-spezifischen Datentypen exportiert und in einem Objektspeicher außerhalb der actinia-Umgebung gespeichert werden. 

Bei beiden Verarbeitungstypen besteht nur Lesezugriff auf alle Karten des verwendeten persistenten Datenbankspeichers.

#### Erstellen einer Prozesskette Schritt für Schritt

Wir erstellen nun Schritt für Schritt eine Prozesskette anhand des Beispiels der Berechnung des Normalized Difference Vegetation Index (NDVI).

Erstellen Sie zunächst eine leere Prozesskette:

In [14]:
process_chain = {"version": 1, "list": []}

Fügen Sie den ersten Eintrag in die Prozesskettenliste ein.

Für Rasteroperationen müssen wir die **Rechenregion** ([computational region](https://grasswiki.osgeo.org/wiki/Computational_region)) auf die Region von Interesse mit der gewünschten Auflösung setzen.

***Hinweis:*** Sie müssen den Namen der Karte und der Mapset folgendermaßen angeben: "map_name@mapset_name".

In [15]:
# list item for g.region
region_process = {
  "id": "g_region_to_dop",
  "module": "g.region",
  "inputs": [
      {
          "param": "raster",
          "value": "dop_nir@PERMANENT"
      }
  ],
  "flags": "p"
}
process_chain["list"].append(region_process)
print_as_json(process_chain)

{
  "version": 1,
  "list": [
    {
      "id": "g_region_to_dop",
      "module": "g.region",
      "inputs": [
        {
          "param": "raster",
          "value": "dop_nir@PERMANENT"
        }
      ],
      "flags": "p"
    }
  ]
}


Nun fügen wir die NDVI-Verarbeitung in die Prozesskettenliste ein:

In [16]:
# list item for r.mapcalc
ndvi_process = {
  "id": "r_mapcalc_ndvi",
  "module": "r.mapcalc",
  "inputs": [
      {
          "param": "expression",
          "value": "ndvi = int(127.5 * ( float((dop_nir@PERMANENT - dop_red@PERMANENT) / (dop_nir@PERMANENT + dop_red@PERMANENT)) + 1.0 ) )"
      }
  ]
}
process_chain["list"].append(ndvi_process)
print_as_json(process_chain)

{
  "version": 1,
  "list": [
    {
      "id": "g_region_to_dop",
      "module": "g.region",
      "inputs": [
        {
          "param": "raster",
          "value": "dop_nir@PERMANENT"
        }
      ],
      "flags": "p"
    },
    {
      "id": "r_mapcalc_ndvi",
      "module": "r.mapcalc",
      "inputs": [
        {
          "param": "expression",
          "value": "ndvi = int(127.5 * ( float((dop_nir@PERMANENT - dop_red@PERMANENT) / (dop_nir@PERMANENT + dop_red@PERMANENT)) + 1.0 ) )"
        }
      ]
    }
  ]
}


Anzeige der Statistikinformationen der berechneten NDVI-Karte:

In [17]:
# list item for r.univar
ndvi_stats_process = {
          "id": "r_univar_ndvi",
          "module": "r.univar",
          "inputs": [
              {
                  "param": "map",
                  "value": "ndvi"
              }
          ],
          "flags": "g",
          "stdout": {"id": "ndvi_stats", "format": "kv", "delimiter": "="}
      }
process_chain["list"].append(ndvi_stats_process)
print_as_json(process_chain)

{
  "version": 1,
  "list": [
    {
      "id": "g_region_to_dop",
      "module": "g.region",
      "inputs": [
        {
          "param": "raster",
          "value": "dop_nir@PERMANENT"
        }
      ],
      "flags": "p"
    },
    {
      "id": "r_mapcalc_ndvi",
      "module": "r.mapcalc",
      "inputs": [
        {
          "param": "expression",
          "value": "ndvi = int(127.5 * ( float((dop_nir@PERMANENT - dop_red@PERMANENT) / (dop_nir@PERMANENT + dop_red@PERMANENT)) + 1.0 ) )"
        }
      ]
    },
    {
      "id": "r_univar_ndvi",
      "module": "r.univar",
      "inputs": [
        {
          "param": "map",
          "value": "ndvi"
        }
      ],
      "flags": "g",
      "stdout": {
        "id": "ndvi_stats",
        "format": "kv",
        "delimiter": "="
      }
    }
  ]
}


Export der NDVI-Rasterkarte im COG-Format, da wir im ephemeralen Modus gerechnet haben:

In [18]:
# list item for exporter
export_process = {
  "id": "exporter_ndvi",
  "module": "exporter",
  "outputs": [
    {
      "export": {
        "type": "raster",
        "format": "COG"
      },
      "param": "map",
      "value": "ndvi"
    }
  ]
}
process_chain["list"].append(export_process)
print_as_json(process_chain)

{
  "version": 1,
  "list": [
    {
      "id": "g_region_to_dop",
      "module": "g.region",
      "inputs": [
        {
          "param": "raster",
          "value": "dop_nir@PERMANENT"
        }
      ],
      "flags": "p"
    },
    {
      "id": "r_mapcalc_ndvi",
      "module": "r.mapcalc",
      "inputs": [
        {
          "param": "expression",
          "value": "ndvi = int(127.5 * ( float((dop_nir@PERMANENT - dop_red@PERMANENT) / (dop_nir@PERMANENT + dop_red@PERMANENT)) + 1.0 ) )"
        }
      ]
    },
    {
      "id": "r_univar_ndvi",
      "module": "r.univar",
      "inputs": [
        {
          "param": "map",
          "value": "ndvi"
        }
      ],
      "flags": "g",
      "stdout": {
        "id": "ndvi_stats",
        "format": "kv",
        "delimiter": "="
      }
    },
    {
      "id": "exporter_ndvi",
      "module": "exporter",
      "outputs": [
        {
          "export": {
            "type": "raster",
            "format": "COG"
        

Führen Sie nun den Job aus, indem Sie die Prozesskette an den ephemeren Endpunkt `/locations/<location_name>/processing_async_export` senden.

In [19]:
# create a POST request to the Actinia Data API
request_url = actinia_url + "/locations/fossgis2023_epsg25832_utm32N/processing_async_export"
request = requests.post(url=request_url, auth=actinia_auth, json=process_chain)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()
print("Response with status code %d:" % request.status_code)

# print formatted JSON
print_as_json(jsonResponse)

# status url
request_url = jsonResponse["urls"]["status"]
print("status url:")
print(request_url)

Response with status code 200:
{
  "accept_datetime": "2023-03-03 08:43:16.938511",
  "accept_timestamp": 1677832996.9385076,
  "api_info": {
    "endpoint": "asyncephemeralexportresource",
    "method": "POST",
    "path": "/api/v3/locations/fossgis2023_epsg25832_utm32N/processing_async_export",
    "request_url": "http://actinia.mundialis.de/api/v3/locations/fossgis2023_epsg25832_utm32N/processing_async_export"
  },
  "datetime": "2023-03-03 08:43:16.941115",
  "http_code": 200,
  "message": "Resource accepted",
  "process_chain_list": [],
  "process_results": {},
  "queue": "local",
  "resource_id": "resource_id-2d6704d9-fdf2-4c64-93e4-0fa053d70bde",
  "status": "accepted",
  "time_delta": 0.002614736557006836,
  "timestamp": 1677832996.9411137,
  "urls": {
    "resources": [],
    "status": "https://actinia.mundialis.de/api/v3/resources/fossgis2023/resource_id-2d6704d9-fdf2-4c64-93e4-0fa053d70bde"
  },
  "user_id": "fossgis2023"
}
status url:
https://actinia.mundialis.de/api/v3/res

Das Wichtigste an der Actinia-Antwort ist der **Status**, der `accepted` oder `running` sein sollte, und die **Status-URL** unter `urls - status`.

Die Verarbeitung erfolgt asynchron, was bedeutet, dass die Anfrage gesendet wird und Sie nur die Status-URL erhalten, so dass Sie den aktuellen Status der Verarbeitung abfragen können, bis der Job beendet ist.

Der **Status** eines Prozesses kann sein:
* accepted: actinia hat den Auftrag erhalten und wird in Kürze mit der Verarbeitung beginnen
* running: actinia führt den Auftrag aus
* finished: actinia hat den Auftrag erfolgreich beendet
* error: während der Ausführung des Auftrags ist ein Fehler aufgetreten
* terminated: ein: Benutzer:in hat den Auftrag abgebrochen

Auftrag anfordern bis zum Abschluss oder Fehler:

In [20]:
# continue polling until finished
print(request_url)

while request.status_code == 200 and \
        jsonResponse["message"] != "Processing successfully finished":
    request = requests.get(url=request_url, auth=actinia_auth)
    jsonResponse = request.json()

# check if anything went wrong
verify_request(request, 200)
    
# print formatted JSON
print_as_json(jsonResponse)

https://actinia.mundialis.de/api/v3/resources/fossgis2023/resource_id-2d6704d9-fdf2-4c64-93e4-0fa053d70bde
{
  "accept_datetime": "2023-03-03 08:43:16.938511",
  "accept_timestamp": 1677832996.9385076,
  "api_info": {
    "endpoint": "asyncephemeralexportresource",
    "method": "POST",
    "path": "/api/v3/locations/fossgis2023_epsg25832_utm32N/processing_async_export",
    "request_url": "http://actinia.mundialis.de/api/v3/locations/fossgis2023_epsg25832_utm32N/processing_async_export"
  },
  "datetime": "2023-03-03 08:43:31.462777",
  "http_code": 200,
  "message": "Processing successfully finished",
  "process_chain_list": [
    {
      "list": [
        {
          "flags": "p",
          "id": "g_region_to_dop",
          "inputs": [
            {
              "param": "raster",
              "value": "dop_nir@PERMANENT"
            }
          ],
          "module": "g.region"
        },
        {
          "id": "r_mapcalc_ndvi",
          "inputs": [
            {
           

Visualisierung der mit Actinia berechneten Karten in Leafmap:

In [27]:
import leafmap

ndvi_url = jsonResponse["urls"]["resources"][0]

# add the credentials to URL

ndvi_url = ndvi_url.replace("//", "//fossgis2023:ieh0ahweefavicieca6g@")

Prüfen, ob es sich um das COG Format handelt (sollte `True` liefern; Analyse dauert einen Moment):

In [28]:
leafmap.cog_validate(ndvi_url)

(True, [], [])

Anzeigen mit der mit actinia berechneten Karte in Leafmap:

<span style="color:red">**TODO oben color=ndvi einbauen!**</span>

In [32]:
m = leafmap.Map()

# colors as hex or RGB values,
# color can be defined using either hex code or RGB (0-255, 0-255, 0-255)
# see: https://leafmap.org/notebooks/06_legend/ and https://leafmap.org/notebooks/07_colorbar/
# default_vis=True: use color table of COG file\n",

m.add_cog_layer(ndvi_url, name="NDVI", default_vis=True,
                attribution='<a href="https://actinia.mundialis.de/">https://actinia.mundialis.de/</a>')
# show map
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

Die Karte zeigt die NDVI-Werte aus Sentinel-2 am Überflugstermin des Satelliten an.

Hinweis: Leafmap bietet oben rechts eine reichhaltige Toolbox.